In [201]:
import pandas as pd
import os
import datetime as dt

## rows 500
pd.set_option('display.max_rows', 500)

## columns
pd.set_option('display.max_columns', 500)

# 당기 고정자산명세 정리 기본

당기 = 2023
기말 = pd.to_datetime((str(당기)+'-12-31'))

df = pd.read_excel('202312_고정자산명세서.xlsx')

df = df[df['자산유형'] != '소계']
df = df[df['자산유형'] != '합계']
df = df[~(df['자산상태'] == '상각불가')]
df = df[~((df['자산유형'] == '건물') & (df['자산상태'] == '폐기') & (df['내용년수'] == 37))]


cond_1 = ['자산유형','자산명','코스트센터','자산상태','취득일자','취득가액','내용년수','상각방법','상각율',\
         '기초가액','전월말상각누계액','기말가액','일반상각비','당기말상각누계액','전월말손상누계액','당월말손상누계액','당월말장부가액']
df['취득일자'] = pd.to_datetime(df['취득일자'])

전기_말일 = pd.to_datetime(str(df['취득일자'].sort_values(ascending=False).iloc[0].year - 1) + '-' +'12-31')

df = df[cond_1]
df['귀속월'] = df['취득일자'].dt.month.astype(float)

년차 = [기말.year - i.year for i in df['취득일자']]
df['년차'] = 년차

# 손상여부 구분

df.loc[(df['당월말손상누계액'] != 0) & (df['당월말손상누계액'].notnull()), '손상여부'] = '손상'
df.loc[df['손상여부'] != '손상','손상여부'] = '정상'
# df.groupby('손상여부').agg({'취득가액':'sum'})   # 손상, 비손상 금액확인

# 자산 항목별 구분 : 건물 & 구축물 & 손상자산

c1 = df['자산유형'] == '건물'
c2 = df['취득일자'] <= 전기_말일

df.loc[c1 & c2, '구분'] = '건물_전기'    # 취득일자에 따른 전기, 당기 구분
c2 = ~(df['취득일자'] <= 전기_말일)
df.loc[c1 & c2, '구분'] = '건물_당기'

c1 = (df['자산유형'] == '구축물') & \
        (df['취득일자'] <= 전기_말일) & \
        (df['당월말손상누계액'] == 0) & \
        (df['내용년수'] == 20) & \
        (df['자산상태'] == '상각진행')

df.loc[c1,'구분'] = '구축물_전기'

c1 = (df['자산유형'] == '구축물') & \
        (df['취득일자'] <= 전기_말일) & \
        (df['당월말손상누계액'] == 0) & \
        (df['내용년수'] == 15)
df.loc[c1,'구분'] = '구축물_비트'

c1 = (df['자산유형'] == '구축물') & \
        (df['자산상태'] == '상각진행') & \
        (df['취득일자'].dt.year == 당기)

# df['귀속월'] = df.loc[c1]['취득일자'].dt.month.astype(str)
# df.loc[c1,'구분'] = '구축물_당기_' + df['귀속월'].astype(str)
df.loc[c1,'구분'] = '구축물_당기' 

c1 = (df['자산유형'] == '구축물') & \
        (df['취득일자'] <= 전기_말일) & \
        (df['당월말손상누계액'] > 0) & \
        (df['내용년수'] == 20) & \
        (df['코스트센터'] == 'xxx점(오프라인)')

df.loc[c1,'구분'] = '구축물_xxx_손상'

c1 = (df['자산유형'] == '구축물') & \
        (df['취득일자'] <= 전기_말일) & \
        (df['당월말손상누계액'] > 0) & \
        (df['내용년수'] == 20) & \
        (df['코스트센터'] == '본사(오프라인)')
df.loc[c1,'구분'] = '구축물_할라고_손상'

c1 = (df['자산유형'] == '시설장치') & \
        (df['취득일자'] <= 전기_말일) & \
        (df['당월말손상누계액'] > 0) & \
        (df['내용년수'] == 5) & \
        (df['코스트센터'] == '본사(오프라인)')
df.loc[c1,'구분'] = '시설장치_할라고_손상'

c1 = (df['자산유형'] == '집기비품') & \
        (df['취득일자'] <= 전기_말일) & \
        (df['당월말손상누계액'] > 0) & \
        (df['내용년수'] == 5) & \
        (df['코스트센터'] == '본사(오프라인)')
df.loc[c1,'구분'] = '집기비품_할라고_손상'

In [208]:
# 차량운반구 구분 : 차량별


# 자산별 구분할 때는 전체를 먼저
c1 = (df['자산유형'] == '차량운반구') & \
        (df['자산상태'] == '상각완료')
df.loc[c1, '구분'] = '차량운반구_상각완료'


c1 = (df['자산유형'] == '차량운반구') & \
        (df['자산상태'] == '상각진행') & \
        (df['구분'].isnull())
df.loc[c1, '구분'] = '차량운반구_상각진행'


# 합계표 만들 때는 세부를 먼저 후 전체구분 진행
c1 = (df['자산유형'] == '차량운반구') & \
        (df['취득일자'].dt.year == 2018) & \
        (df['취득일자'].dt.month == 10)

df.loc[c1, '구분'] = '차량운반구_0954'

c1 = (df['자산유형'] == '차량운반구') & \
        (df['취득일자'].dt.year == 2022) & \
        (df['취득일자'].dt.month == 10)

df.loc[c1, '구분'] = '차량운반구_7690'

c1 = (df['자산유형'] == '차량운반구') & \
        (df['취득일자'].dt.year == 2022) & \
        (df['취득일자'].dt.month == 11)

df.loc[c1, '구분'] = '차량운반구_8084'



# 차량운반구 : 당기상각시인범위액 계산
### 법인세법상 차량운반구 감가상각은 정액법 적용

c1 = (df['자산유형'] == '차량운반구')
c2 = df['자산명'].str.contains('0954|7690|8084')

df.loc[c1 & c2,'상각방법'] = '정액법'
df.loc[c1 & c2,'상각율'] = 1 / df.loc[c1 & c2,'내용년수']

df.loc[c1&c2,'당기상각시인범위액'] = round(df.loc[c1&c2,'취득가액'] * df.loc[c1&c2,'상각율'],0)

# 집기비품

c1 = (df['자산유형'] == '집기비품') & \
        (df['자산상태'] == '상각완료')

df.loc[c1,'구분'] = '집기비품_상각완료'

c1 = (df['자산유형'] == '집기비품') & \
        (df['자산상태'] == '상각진행') & \
        (df['취득일자'] <= 전기_말일) & \
        (df['당월말손상누계액'] == 0)

df.loc[c1,'구분'] = '집기비품_전기'

c1 = (df['자산유형'] == '집기비품') & \
        (df['자산상태'] == '상각진행') & \
        (df['취득일자'].dt.year == 당기)
# df['귀속월'] = df.loc[c1]['취득일자'].dt.month.astype(str)
# df.loc[c1,'구분'] = '집기비품_당기_' + df['귀속월'].astype(str)
df.loc[c1,'구분'] = '집기비품_당기'

# 할라고 손상분 별도 구분
c1 = (df['자산유형'] == '집기비품') & \
        (df['자산상태'] == '상각완료') & \
        (df['당월말손상누계액'] > 0)
df.loc[c1, '구분'] = '집기비품_손상'

# 시설장치

c1 = (df['자산유형'] == '시설장치') & \
        (df['자산상태'] == '상각완료') & \
        (df['당월말손상누계액'] == 0)
df.loc[c1,'구분'] = '시설장치_상각완료'

c1 = (df['자산유형'] == '시설장치') & \
        (df['자산상태'] == '상각진행') & \
        (df['취득일자'] <= 전기_말일) & \
        (df['당월말손상누계액'] == 0)
df.loc[c1,'구분'] = '시설장치_전기'

c1 = (df['자산유형'] == '시설장치') & \
        (df['자산상태'] == '상각진행') & \
        (df['취득일자'].dt.year == 당기)

# df['귀속월'] = df.loc[c1]['취득일자'].dt.month.astype(str)
# df.loc[c1,'구분'] = '시설장치_당기_' + df['귀속월'].astype(str)
df.loc[c1,'구분'] = '시설장치_당기'

# 할라고 손상분 별도 구분
c1 = (df['자산유형'] == '시설장치') & \
        (df['자산상태'] == '상각완료') & \
        (df['당월말손상누계액'] > 0)
df.loc[c1, '구분'] = '시설장치_손상'

# 무형자산 구분

c1 = (df['일반상각비'] > 0) & \
        (df['자산유형'] == '영업권')

df.loc[c1,'구분'] = '영업권'

c1 = (df['자산상태'] == '상각완료') & \
        (df['일반상각비'] == 0) & \
        (df['자산유형'] == '영업권')

df.loc[c1,'구분'] = '영업권_완료'


c1 = (df['자산상태'] == '상각진행') & \
        (df['자산유형'] == '사용수익기부자산')
df.loc[c1,'구분'] = '사용수익기부'

c1 = (df['자산상태'] == '상각완료') & \
        (df['자산유형'] == '사용수익기부자산')
df.loc[c1,'구분'] = '사용수익기부_완료'


# 당기는 월별로 집계하기 ======

c1 = (df['자산상태'] == '상각진행') & \
        (df['자산유형'] == '기타무형자산_기타') & \
        (df['취득일자'].dt.year == 당기)
# df['귀속월'] = df.loc[c1]['취득일자'].dt.month.astype(str)
# df.loc[c1,'구분'] = '기타무형자산_당기_' + df['귀속월'].astype(str)
df.loc[c1,'구분'] = '기타무형자산_당기'


#  무형자산_전기_상각진행

c1 = (df['자산상태'] == '상각진행') & \
        (df['자산유형'] == '기타무형자산_기타') & \
        (df['취득일자'].dt.year < 당기)

df.loc[c1, '구분'] = '기타무형자산_전기'

#  무형자산_전기_상각완료

c1 = (df['자산상태'] == '상각완료') & \
        (df['자산유형'] == '기타무형자산_기타') & \
        (df['일반상각비'] > 0) & \
        (df['취득일자'].dt.year < 당기)

df.loc[c1, '구분'] = '기타무형자산_전기'

# 무형자산_전기_미술품 ======
c1 = (df['기초가액'] == df['당월말장부가액']) & \
        (df['자산유형'] == '기타무형자산_기타') & \
        (df['코스트센터'].str.contains('xxx'))

df.loc[c1, '구분'] = '기타무형자산_xxx_완료'


df.loc[(df['구분'].notnull()) & (df['구분'].str.contains('무형')),'상각방법'] = '정율법'
df.loc[(df['구분'].notnull()) & (df['구분'].str.contains('무형')),'상각율'] = 0.451

# 세부구분시 : 당기 취득분은 취득월을 표기
# 합계표 작성시 : 취득월 제외 후 합산

df.loc[df['취득일자'].dt.year == 당기,'구분'] = df.loc[df['취득일자'].dt.year == 당기,'구분'] + '_' +df.loc[df['취득일자'].dt.year == 당기,'취득일자'].dt.month.astype(str)

# 1년 미만 자산 -> 월할계산

one_year = dt.timedelta(days=365)

c1 = df['상각방법'] == '정액법'
df.loc[c1,'일수']  = 기말 - df.loc[c1,'취득일자']

c3 = df['일수'] < one_year

df.loc[c1&c3, '당기상각시인범위액'] = round(df.loc[c1&c3,'취득가액'] * df.loc[c1&c3, '상각율'] * (기말.month - df.loc[c1&c3, '귀속월'] + 1) / 12, 0)

# 상각 + 손상 합계 계산

df.loc[:,'상각_손상'] = df.loc[:,'당기말상각누계액'] + df.loc[:,'당월말손상누계액']

## 정율법 정리

In [219]:
# 정율법 전체 적용 부분

# 손상일 때만 전기말 부인누계 있음

# 전기말 부인누계액 만들기
c1 = (df['상각방법'] == '정율법') & (df['손상여부'] == '손상')
df.loc[c1, '전기말부인_누계'] = df.loc[c1, '취득가액'] * (1 - df.loc[c1, '상각율']) ** df.loc[c1, '년차']
df.loc[(df['상각방법'] == '정율법'),'전기말부인_누계'] = df.loc[(df['상각방법'] == '정율법'),'전기말부인_누계'].fillna(0)

# 당기상각 시인범위액 계산
df.loc[(df['상각방법'] == '정율법'), '가감계'] = df.loc[(df['상각방법'] == '정율법'), '일반상각비'] + df.loc[(df['상각방법'] == '정율법'), '당월말장부가액'] + df.loc[(df['상각방법'] == '정율법'), '전기말부인_누계'] 
df.loc[(df['상각방법'] == '정율법'), '기존_상각비'] = df.loc[(df['상각방법'] == '정율법'), '가감계'] * df.loc[(df['상각방법'] == '정율법'), '상각율']
df.loc[(df['상각방법'] == '정율법'),'당기상각시인범위액'] = df.loc[(df['상각방법'] == '정율법'), '기존_상각비']


# 정율법 : 당기상각시인액 계산 : 0 년차 월할계산

c2 = (df['상각방법'] == '정율법') & (df['년차'] == 0)
df.loc[c2,'당기상각시인범위액'] = round(df.loc[c2,'가감계'] * df.loc[c2,'상각율'] * (기말.month - df.loc[c2,'귀속월'] + 1) / 12,0)

c2 = (df['상각방법'] == '정율법') & (df['년차'] != 0)
df.loc[c2,'당기상각시인범위액'] = round(df.loc[c2,'가감계'] * df.loc[c2,'상각율'],0)


# 손상자산 적용
c1 = (df['상각방법'] == '정율법') & (df['손상여부'] == '손상')
df.loc[c1, '기왕부인액중당기손금'] = df.loc[c1,'당기상각시인범위액'] - df.loc[c1,'일반상각비']

# 상각완료 자산
c1 = (df['상각방법'] == '정율법') & (df['구분'].str.contains('상각완료'))
df.loc[c1, '당기상각시인범위액'] = df.loc[c1, '일반상각비'] + df.loc[c1, '당월말장부가액']

# 무형자산
c1 = (df['상각방법'] == '정율법') & (df['자산유형'] == '기타무형자산_기타') & (df['년차'] == 0)
df.loc[c1, '당기상각시인범위액'] = round(df.loc[c1,'가감계'] * df.loc[c1,'상각율'] * (기말.month - df.loc[c1,'귀속월'] + 1) / 12,0)

c1 = (df['상각방법'] == '정율법') & (df['구분'] == '기타무형자산_전기')
df.loc[c1, '당기상각시인범위액'] = round(df.loc[c1,'가감계'] * df.loc[c1,'상각율'],0)

In [225]:
# 구분값이 있는 데이터만 추출 (=상각진행 데이터)

c1 = df['구분'].notnull()
c2 = df['자산상태'] != '매각'

df_2 = df[c1 & c2]


df_3 = pd.pivot_table(df_2,index=['구분', '내용년수'], values=['기말가액','상각_손상','당월말장부가액','전월말상각누계액',
                                               '일반상각비','취득가액','당기상각시인범위액','기왕부인액중당기손금'],aggfunc='sum')

df_3['당기말누계액'] = df_3['전월말상각누계액'] + df_3['일반상각비']

# 구분된 자산별 내용년수 상각방법 상각율 테이블

dep_table = df_2[['구분','내용년수','상각방법','상각율']].drop_duplicates().set_index('구분')
dep_table.loc[dep_table.index.str.contains('무형'),'상각방법'] = '정율법'

In [226]:
df_3.loc[df_3.index.get_level_values('구분').str.contains('상각완료'),'당기상각시인범위액'] = df_3.loc[df_3.index.get_level_values('구분').str.contains('상각완료'),'당기상각시인범위액'] - 1000

In [227]:
# 상각진행 자산항목과 상각방법 테이블 MERGE

cols = ['기말가액','상각_손상','당월말장부가액','전월말상각누계액','일반상각비','당기말누계액','취득가액','상각율','상각방법','내용년수','당기상각시인범위액','기왕부인액중당기손금']
df_4 = pd.merge(df_3, dep_table, on=['구분', '내용년수'])[cols]
df_4 = df_4.rename(columns = {'기말가액':'1.기말현재액',
                       '상각_손상':'2.감가상각누계액',
                       '당월말장부가액':'3.미상각잔액',
                       '전월말상각누계액':'4.전기말누계',
                       '일반상각비':'5.당기상각비',
                       '당기말누계액':'6.당기말누계액',
                       '취득가액':'7.취득가액',
                       '상각율':'8.상각율',
                       '상각방법':'0.상각방법',
                       '내용년수':'0.내용년수',
                       '당기상각시인범위액' : '9.당기상각시인범위액',
                       '기왕부인액중당기손금' : '10.기왕부인액중당기손금'})

In [228]:
# 정액법 & 손상 케이스

c1 = df_4.index.str.contains('손상')
c2 = df_4['9.당기상각시인범위액'] == 0
c3 = df_4['0.상각방법'] == '정액법'

df_4.loc[c1 & c2 & c3,'9.당기상각시인범위액'] = round(df_4.loc[c1 & c2 & c3,'1.기말현재액'] / df_4.loc[c1 & c2,'0.내용년수'])

In [229]:
df_5 = df_4.sort_index(axis=1)

In [230]:
# === 정액법 ===

c1 = df_5['0.상각방법'] == '정액법'

# 정액법 일반 케이스 계산
df_5.loc[c1,'9.일반상각액'] = df_5.loc[c1,'5.당기상각비']
df_5.loc[c1,'4.전기말누계'] = df_5.loc[c1,'2.감가상각누계액'] - df_5.loc[c1,'5.당기상각비']

# 당기말 누계액 재계산
df_5.loc[c1,'6.당기말누계액'] = df_5.loc[c1,'4.전기말누계'] + df_5.loc[c1,'5.당기상각비']

# 취득가액 재계산
df_5.loc[c1,'7.취득가액'] = df_5.loc[c1,'3.미상각잔액'] + df_5.loc[c1,'6.당기말누계액']

# 정액법 손상 케이스 계산

df_5.loc[c1 & (df_5['9.당기상각시인범위액'] == 0),'9.당기상각시인범위액'] = df_5.loc[c1 & (df_5['9.당기상각시인범위액'] == 0),'9.일반상각액']
# df_5.loc[c1 & (df_5['9.당기상각시인범위액'] == 0),'9.당기상각시인범위액'] = df_5.loc[c1 & (df_5['9.당기상각시인범위액'] == 0),'7.취득가액'] * df_5.loc[c1 & (df_5['9.당기상각시인범위액'] == 0),'8.상각율']

df_5.loc[c1, '10.기왕부인액중당기손금'] = df_5.loc[c1,'9.당기상각시인범위액'] - df_5.loc[c1,'9.일반상각액']


df_2[df_2['구분'] == '집기비품_전기'].to_excel('temp_집기비품_전기_v1.xlsx')

df_5 = df_5.sort_index(axis=1)

c1 = df_5['0.상각방법'] == '정율법'
df_5[c1].T.to_excel('02.정율법_test_7.xlsx')

c1 = df_5['0.상각방법'] == '정액법'
df_5[c1].T.to_excel('01.정액법_test_7.xlsx')

## 감가상각비 조정명세서 합계표
1. 합계표를 위한 재분류 작업

In [237]:
df_5.loc[df_5.index.str.contains('건물'),'조정명세서'] = '건물'
df_5.loc[(df_5.index == '사용수익기부') | (df_5.index == '영업권'), '조정명세서'] = '무형자산'
df_5.loc[df_5['조정명세서'].isnull(),'조정명세서'] = '기타자산'

In [239]:
df_5.loc[df_5['10.기왕부인액중당기손금'] < 0, '11.상각부인액'] = -df_5.loc[df_5['10.기왕부인액중당기손금'] < 0, '10.기왕부인액중당기손금']
df_5['11.상각부인액'].fillna(0, inplace=True)
df_5.loc[df_5['10.기왕부인액중당기손금'] < 0, '10.기왕부인액중당기손금'] = 0

In [240]:
df_5['12.시인부족액'] = df_5['9.당기상각시인범위액'] - df_5['5.당기상각비'] + df_5['11.상각부인액']

In [241]:
합계표 = df_5.groupby('조정명세서')[['1.기말현재액', '2.감가상각누계액','3.미상각잔액',
                       '9.당기상각시인범위액','5.당기상각비','10.기왕부인액중당기손금',
                       '11.상각부인액','12.시인부족액']].sum().T

In [243]:
합계표.to_excel('03.조정명세서합계표_test7.xlsx')
df_5.to_excel('04.총괄_합계표_test7.xlsx')